In [ ]:
!pip install transformers trl accelerate torch bitsandbytes peft datasets -qU

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.15 requires torch<2.4,>=1.10, but you have torch 2.4.0 which is incompatible.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.


In [ ]:
import os
os.environ['HF_TOKEN'] = "--"

In [ ]:
from datasets import load_dataset

instruct_tune_dataset = load_dataset("FinGPT/fingpt-fiqa_qa")
instruct_tune_dataset

type(instruct_tune_dataset)


Generating train split:   0%|          | 0/17110 [00:00<?, ? examples/s]

datasets.dataset_dict.DatasetDict

In [ ]:
instruct_tune_dataset["train"]

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 17110
})

In [ ]:
instruct_tune_dataset["test"] = instruct_tune_dataset["train"].select(range(3000, 4500))
instruct_tune_dataset["train"] = instruct_tune_dataset["train"].select(range(3000))

In [ ]:
instruct_tune_dataset["train"][0]

{'input': 'What is considered a business expense on a business trip?',
 'output': 'The IRS Guidance pertaining to the subject.  In general the best I can say is your business expense may be deductible.  But it depends on the circumstances and what it is you want to deduct. Travel Taxpayers who travel away from home on business may deduct related   expenses, including the cost of reaching their destination, the cost   of lodging and meals and other ordinary and necessary expenses.   Taxpayers are considered “traveling away from home” if their duties   require them to be away from home substantially longer than an   ordinary day’s work and they need to sleep or rest to meet the demands   of their work. The actual cost of meals and incidental expenses may be   deducted or the taxpayer may use a standard meal allowance and reduced   record keeping requirements. Regardless of the method used, meal   deductions are generally limited to 50 percent as stated earlier.    Only actual costs for l

In [ ]:
instruct_tune_dataset["test"][0]

{'input': 'How much time would I have to spend trading to turn a profit?',
 'output': 'What determines your profitability is not your time, but your TRADES. It is probably a mistake to go into the market and say, I hope to make X% today/this month/this year. As a practical matter, you can make a lot of money in a short period of time, or lose a lot over a long period of time (the latter is more likely). You\'re better off looking at potential trades and saying "I like this trade" (be sure to know why) and "I dislike that trade." If you\'re right about your chosen trade, you\'ll make money. Probably not on your original timetable, because markets react more slowly than individual people do. Then make ONLY those trades that you genuinely like and understand. IF you get into a "rhythm," (rather few people do), your experience might tell you that you are likely to make, say, X% per month or year. But that\'s ONLY if the market continues to accommodate YOUR style of trading. If the markets 

In [ ]:
def create_prompt(sample):
    bos_token = "<s>"
    original_system_message = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    system_message = "Use the provided input, response to generate the response with an LLM."
    input = sample["input"]
    eos_token = "</s>"

    full_prompt = ""
    full_prompt += bos_token
    full_prompt += "\n" + original_system_message + "\n" + system_message + "\n"
    full_prompt += "### Instruction:"
    full_prompt += "\n" + sample['instruction']
    full_prompt += "\n\n### Input:"
    full_prompt += "\n" + sample['input']
    full_prompt += "\n\n### Response:"
    full_prompt += "\n" + sample['output']
    full_prompt += eos_token

    return full_prompt

In [ ]:
create_prompt(instruct_tune_dataset["train"][0])

'<s>\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.\nUse the provided input, response to generate the response with an LLM.\n### Instruction:\nUtilize your financial knowledge, give your answer or opinion to the input question or subject . Answer format is not limited.\n\n### Input:\nWhat is considered a business expense on a business trip?\n\n### Response:\nThe IRS Guidance pertaining to the subject.  In general the best I can say is your business expense may be deductible.  But it depends on the circumstances and what it is you want to deduct. Travel Taxpayers who travel away from home on business may deduct related   expenses, including the cost of reaching their destination, the cost   of lodging and meals and other ordinary and necessary expenses.   Taxpayers are considered “traveling away from home” if their duties   require them to be away from home substantially longer than an   ordinary day’s work and they need to sle

In [ ]:
import torch
from transformers import BitsAndBytesConfig
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2b-it",
    device_map='auto',
    quantization_config=nf4_config,
    use_cache=False
)

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import Trainer, TrainingArguments

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

2024-08-12 11:47:05.893331: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-12 11:47:05.893444: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-12 11:47:06.018735: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
args = TrainingArguments(
  output_dir = "gemma_instruct_generation",
#   num_train_epochs=1,
  max_steps = 50,
  per_device_train_batch_size = 8,
  warmup_steps = 3,
  logging_steps=10,
  save_strategy="epoch",
#   evaluation_strategy="epoch",
  evaluation_strategy="steps",
  eval_steps=10,
  learning_rate=2e-4,
  bf16=True,
  lr_scheduler_type='constant',
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
max_seq_length = 256
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt,
  args=args,
  train_dataset=instruct_tune_dataset["train"],
  eval_dataset=instruct_tune_dataset["test"]
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, packing. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:192: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `S

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:432: UserWarning: You passed `packing=True` to the SFTTrainer/SFTConfig, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [ ]:
import time
start = time.time()
trainer.train()
print(time.time()- start)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss
10,4.991700,3.964801
20,3.652800,3.433610
30,3.249800,3.190021
40,3.124500,2.997518
50,2.897100,2.827067


2481.413826227188


In [ ]:
trainer.save_model("gemma_instruct_generation")

In [ ]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0].replace(prompt, "")

In [ ]:
instruct_tune_dataset_1 = load_dataset("FinGPT/fingpt-fiqa_qa")

generate_response(create_prompt(instruct_tune_dataset_1['train'][6000]), model)

'<bos>\n"How to Deal with Difficult Family In-Laws" and "Financial Independence Defined in Three Simple Steps".<eos>'

Perplexity

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

def calculate_perplexity(model, tokenizer, text):
    encodings = tokenizer(text, return_tensors='pt')
    max_length = model.config.n_positions
    stride = 512
    lls = []

    for i in range(0, encodings.input_ids.size(1), stride):
        begin_loc = max(i + stride - max_length, 0)
        end_loc = min(i + stride, encodings.input_ids.size(1))
        trg_len = end_loc - i  # may be different from stride on last loop
        input_ids = encodings.input_ids[:, begin_loc:end_loc].to(model.device)
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            log_likelihood = outputs.loss * trg_len

        lls.append(log_likelihood)

    ppl = torch.exp(torch.stack(lls).sum() / end_loc)
    return ppl.item()

# Example usage:
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

text = create_prompt(instruct_tune_dataset_1['train'][6000])
perplexity = calculate_perplexity(model, tokenizer, text)
print(f"Perplexity: {perplexity}")


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Perplexity: 85.0977783203125


In [ ]:
perplexity_scores = []
for qi in range(6000, 6050):
    question = instruct_tune_dataset_1['train'][qi]
    text = create_prompt(question)
    perplexity = calculate_perplexity(model, tokenizer, text)
    perplexity_scores += [perplexity]

Token indices sequence length is longer than the specified maximum sequence length for this model (1245 > 1024). Running this sequence through the model will result in indexing errors


NameError: name 'avg' is not defined

In [ ]:
print("Perplexity Scores = ", perplexity_scores)
print("Average Perplexity Score = ", sum(perplexity_scores)/len(perplexity_scores))

Perplexity Scores =  [85.0977783203125, 39.03424835205078, 31.85991096496582, 46.065731048583984, 50.841651916503906, 36.39962387084961, 42.20903396606445, 54.20967483520508, 52.67879104614258, 30.671279907226562, 58.56127166748047, 52.64979553222656, 33.70178985595703, 25.746946334838867, 34.511863708496094, 38.29937744140625, 97.82427978515625, 24.773008346557617, 28.972036361694336, 59.04855728149414, 50.26161193847656, 35.8034553527832, 30.635177612304688, 32.95157241821289, 30.27048683166504, 25.227563858032227, 52.494537353515625, 48.4195671081543, 38.48335647583008, 24.0301513671875, 22.293746948242188, 41.56494903564453, 42.232177734375, 43.23939514160156, 45.46876907348633, 30.482315063476562, 40.90268325805664, 34.24925231933594, 24.274309158325195, 53.2537956237793, 41.27415466308594, 34.88194274902344, 77.32930755615234, 36.610389709472656, 46.761173248291016, 39.182315826416016, 34.78038787841797, 120.95513153076172, 75.3550033569336, 35.39178466796875]
Average Perplexity 